# Implementarea unei Retele Neurale cu un singur strat

In acest exercitiu o sa creati o retea neurala cu un singur strat. Stratul ascuns este conectat la tot inputul si este folosit pentru a clasifica si a testa imagini din datasetul CIFAR-10.

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import cifar10
from keras.datasets import mnist
# initializare pentru matplotlib
plt.rcParams['figure.figsize'] = (10.0, 8.0) # setarea dimensiunii plot-urilor 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# auto-reloading pentru module externe
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ intoarce eroarea relativa """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Definirea retelei

Vom defini reteaua cu un singur layer ascuns cu o clasa. Reteaua primeste ca input x de dimensiunea "input_dim", are un strat ascuns de dimensiune "hidden_dim". Aceasta clasifica input-ul in una din cele "num_classes". 
La iesire, reteaua trece score-urile calculate pe ultimul strat fully_connected printr-o functie **softmax** pentru a scoate probabilitati. 

Functia obiectiv este **cross-entropia**. Adaugam ca regularizare **L2_norm** pe matricile parametrilor (fara bias). Reteaua are o functie non-liniara dupa primul strat fully_connected. Vom folosi **ReLU** pentru aceasta nonliniaritate.

Reteaua are urmatoarea arhitectura: input - fully_connected - ReLU - fully_connected - softmax.
Output-ul celui de-al doilea strat de fully_connected reprezinta score-urile claselor.

Scheletul de cod pentru **ShallowNet** de afla in fisierul **shallow_net.py** pentru a putea lucra in paralel la cele doua fisiere. Alte functii ajutatoare pe care le veti intalni pe parcurs se afla in **utils.py**


# Data de intrare de test
Vom crea o retea simpla cu un singur strat ascuns si vom folosi niste date random pentru a testa corectitudinea implementarii unei propagari inainte si inapoi prin retea

In [ ]:
from shallow_net import ShallowNet
input_dim = 4
hidden_dim = 10
num_classes = 3
dataset_size = 5

np.random.seed(0)
net = ShallowNet(input_dim, hidden_dim, num_classes, std=1e-1)

np.random.seed(1)
X_train = 10 * np.random.randn(dataset_size, input_dim)
y_train = np.array([0, 1, 2, 2, 1])

# Propagare inainte (Forward pass)
## Calcularea scorurilor


In [ ]:
scores = net.loss(X_train)
print(scores)
correct_scores = np.asarray([
  [-0.81233741, -1.27654624, -0.70335995],
  [-0.17129677, -1.18803311, -0.47310444],
  [-0.51590475, -1.01354314, -0.8504215 ],
  [-0.15419291, -0.48629638, -0.52901952],
  [-0.00618733, -0.12435261, -0.15226949]])
# Differenta ar trebui sa fie foarte mica < 1e-7
print(np.sum(np.abs(scores - correct_scores)))

## Calcularea erorii
In aceeasi functie trebuie sa definim partea a doua in care calculam eroare pentru setul de date de test

In [ ]:
loss, _ = net.loss(X_train, y_train, reg=0.05)
correct_loss = 1.30378789133

# diferenta ar trebui sa fie foarte mica < 1e-12
print('Difference between your loss and correct loss:')
print(np.sum(np.abs(loss - correct_loss)))

# Propagare inapoi (Backpropagation)
Implementare completa a functiei loss pentru a cumprinde si gradientul functiei de cost pentru variabilele fc1_w, fc1_b, fc2_w, fc2_b.
Pentru a testa corectitudienea implementarii gradientului putem folosi graientul numeric: (f(x-h) - f(x+h)) / 2h , h -> 0.

In [ ]:
from utils import eval_numerical_gradient

loss, grads = net.loss(X_train, y_train, reg=0.05)
# diferenta ar trebui sa fie foarte mica < 1-8
for param_name in grads:
    f = lambda W: net.loss(X_train, y_train, reg=0.05)[0]
    param_grad_num = eval_numerical_gradient(f, net.params[param_name], verbose=False)
    print('%s max relative error: %e' % (param_name, rel_error(param_grad_num, grads[param_name])))

# Antrenarea retelei (training)
Vom antrena reteaua folosin stochastic gradient descent(SGD). 
In acest pas vom completa functia **train** din clasa ShallowNet. Vom completa de asemenea si functia **predict** pentru a afisa acuratetea retelei in timpul antrenarii.

In [ ]:
stats = net.train(X_train, y_train, X_train, y_train, 
                  learning_rate=1e-1, reg=5e-6,
                  num_iters=100, verbose=False)
print('Final training loss: ', stats['loss_history'][-1])

# plotarea costului
plt.plot(stats['loss_history'])
plt.xlabel('iteration')
plt.ylabel('training loss')
plt.title('Training Loss history')
plt.show()

# Incarcarea datasetului - CIFAR-10
Vom incarca datasetul folosindu-ne de **keras**. 

**Keras** este un wrapper peste **tensorflow** **(framework de deep learning)**. In laboaratoarele viitoare veti folosi tensorflow pentru a scrie retele. Tensorflow are scrise deja biblioteci de functii in C++ ce folosesc rutine optimizate pentru a rula cod cuda pe GPU. Pe acestea le veti apela direct din python. 

In laboratorul de astazi incercam sa implementam totul de mana direct in python pentru a intelege "magic"-ul din spatele Tensorflow-ului.

**CIFAR-10** este un dataset ce contine 50000 de imagini de train si 10000 imagini de test. Acestea eu dimensiuni 32x32x3. O sa spargem imaginile de train in doua splituri pentru a pastra 10000 de imagini pentru validare

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_val = X_train[40000:]
X_train = X_train[:40000]
y_val = y_train[40000:]
y_train = y_train[:40000]

Imaginile vor fi **vectorizate** inainte de a fi trimise retelei.

**Numarul de clase din CIFAR-10** este (big surprise here!) 10. 

### Preprocesarea datasetului
La acest pas, trebuie sa faceti o preprocesare minimala, normalizarea fiecarei imagini scazand valoarea medie peste tot datasetul. Imaginile sunt matrixi 32x32x3 cu tipul uint8, iar noi avem nevoie vectori de tip float32.


In [ ]:
def preprocess_dataset(X_train, y_train, X_val, y_val, X_test, y_test):
    X_train = X_train.reshape(y_train.shape[0], -1)
    X_test = X_test.reshape(y_test.shape[0], -1)
    X_val = X_val.reshape(y_val.shape[0], -1)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_val = X_val.astype('float32')
    y_train = y_train.reshape(y_train.shape[0])
    y_test = y_test.reshape(y_test.shape[0])
    y_val = y_val.reshape(y_val.shape[0])
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_test -= mean_image
    X_val -= mean_image
    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = preprocess_dataset(X_train, y_train, X_val, y_val, X_test, y_test)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Val data shape: ', X_val.shape)
print('Val labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

## Antrenarea retelei pe CIFAR-10
Pentru a antrena reteaua folosim SGD. 

In [ ]:
input_size = 32
input_channels = 3
input_dim = input_channels * input_size * input_size
hidden_dim = 50
num_classes = 10
net = ShallowNet(input_dim, hidden_dim, num_classes)
# Antrenam reteaua
stats = net.train(np.concatenate([X_train, X_test], 0), np.concatenate([y_train, y_test], 0), X_val, y_val,
            num_iters=1000, batch_size=200,
            learning_rate=1e-4, learning_rate_decay=0.95,
            reg=0.25, verbose=True)

# Facem preziceri pe datasetul de test si calculam acuratetea
test_acc = (net.predict(X_val) == y_val).mean()
print('Test accuracy: ', test_acc)



# Debug procesul de antrenare
Test accuracy ar trebui sa fie cu parametrii de mai sus undeva la 0.28. Aceasta este o acuratete foarte mica, random ar fi undeva la 0.10, deci nu suntem foarte departe.
Pentru a vedea ce sa intampla cu antrenarea putem plota valorile erorii pentru antrenare si test

In [ ]:
# Plot the loss function and train / validation accuracies
plt.subplot(2, 1, 1)
plt.plot(stats['loss_history'])
plt.title('Loss history')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.subplot(2, 1, 2)
plt.plot(stats['train_acc_history'], label='train')
plt.plot(stats['val_acc_history'], label='val')
plt.title('Classification accuracy history')
plt.xlabel('Epoch')
plt.ylabel('Clasification accuracy')
plt.show()

In [ ]:
from utils import visualize_grid

# Vizualizarea parametrilor retelei

def show_net_weights(net, param_key):
    W1 = net.params[param_key]
    W1 = W1.reshape(32, 32, 3, -1).transpose(3, 0, 1, 2)
    plt.imshow(visualize_grid(W1, padding=3).astype('uint8'))
    plt.gca().axis('off')
    plt.show()

show_net_weights(net, 'fc1_w')

# Tunarea hiperparametrilor
Cateva observatii:
* costul scade liniar ceea ce ar putea sugera ca putem mari rata de invatare
* nu exista niciun gap intre plotul accuratetii pe datale de invatare vs datele de test ceea ce ar putea sugera ca suntem in regimul de 'underfitting' si avem un model cu capacitate prea mica.
* pentru a dezvolta o intuitie de ce hiperparametrii merg in ce situatii, trebuie experimentat mult. Puteti experimenta cu diferite valori pentru urmatorii hiperparametrii: numarul de neuroni pe stratul ascuns (hidden_dim), rata de invatare etc. 
* in principiu ne dorim o acuratete pe datale de test > 48%.
* pentru a ajunge la o acuratete foarte mare, sunt foarte multe trick-uri pe care le vom invata in episoadele urmatoare.

In [ ]:
best_net = net # cel mai bun model

#################################################################################
# TODO: Tunarea hiperparametrilor pe datele de validare.                        #
# Cel mai bun model trebuie stocat in best_net.                                 #
# Hint: Cea mai simpla tunare poate fi o iterare prin mai multe valori ale      #                            #
# parametrilor pentru hidden_dim, learning_rate, l2_reg, etc.  Departajarea     #
# celui mai bun model se poate face dupa acuratete                              #
#################################################################################
pass
#################################################################################
#                               END OF YOUR CODE                                #
#################################################################################

In [ ]:
# visualize the weights of the best network
show_net_weights(best_net, 'fc1_w')

# Rularea pe datele de test 
Testul final al unei retele este rularea acesteia pe datele de test. De obiecei acestea sunt niste date pastrate deoparte pentru care nu avem label-urile/etichetele. De exemplu, in cadrul unei competitii, se vor publica datele de antrenare cu label-urile corespunzatoare acestora pentru antrenarea unui model, datele de validare cu label-urile corespunzatoare acestora pentru tunarea hiperparametrilor dupa antrenare, si datele de testare fara label-uri. Submisia in cardul unei competitii este facuta cu prezicerile modelului pentru datele de test.

In cazul de fata, noi vom fi si evaluatori, avand deja etichetele reale, putem sa ne calculam singuri acuratetea.

In [ ]:
test_acc = (best_net.predict(X_test) == y_test).mean()
print('Test accuracy: ', test_acc)